# MLlab_LSI_query

This file contains an implementation of an information query method, based on Latent Semantic Indexing (LSI) technique. LSI is a mature and widely used technique in natural language processing. It uses a rank-reduced Singular Value Decomposition( SVD) on TF-IDF matrix ( introduction of TF-IDF matrix is in file TFIDF_company_chart). LSI considers words which occur in the same document tends to have a relationship, and this rank-reduced  SVD can merge these words with similar meaning or strong relationship. One advantage of information query method based on LSI is that it can search highly related contents, even though the words in the query may not appear in these contents. Following are some Mathematics details.


The procedure is:

   1. We at first load 1000 clean documents, and use function TfidfVectorizer from sklearn library to build their TF-IDF matrix.  

   2. Given the query string, we look it as a new document and also get its TF-IDF vector( since only one document) according to the TF-IDF matrix we get in step 1. Here we assume all input queries do not contain repeat words. Therefore for every word in the query, we simply find its correspondent index in TF-IDF matrix and assign its value as $1 * IDF(index)$. And for other words which do not in the query, we simply assign them as $0$.  

   3. Apply rank reduced SVD at TF-IDF matrix, and we get:
$$A \approx A_{k} = U_{k}S_{k}V_k^T$$
     
    Here this $A$ is the original TF-IDF matrix, and $A_{k}$ is its $k$ dimension approximation. We can simply get this $A_{k}$ by evaluating the formula above. Here we can get $S_{k}$, $U_{k}$ and $V_{k}$ by letting the singular value matrix $S$ only preserve $k$ max singular value( $S_{k}$), and let left singular vector matrix $U$, right singular vector matrix $V$ only preserve their first $k$ row vectors and $k$ col vectors, respectively.
    In the code, we simply assign $k = 100$ and use scipy's sparse SVDs function to evaluate this formula efficiently. After this step, we actually approximate this TF-IDF matrix in a low dimension space.  
    
   4. To compute the similarity of query and document vectors in low dimension space, we should also transfer the query vector from step 2 to this low dimension space. We do that by evaluating this formula:
$$q_{k} = S^{-1}_{k}U^T_kq$$

    Here $q_{k}$ is query's $k$ dimension approximation. And $q$ is its original vector. This works because exactly every col vector in the right singular matrix $V$ describe one document, and in LSI technique we also see the query as a new document.  
    
   5. Finally, we use cosine similarities to compute the similarities between the query vector and documents vectors. And take the top 10 similar documents as result. The cosine similarities formula is:
$$sim(q,d) = \frac{q*d}{\begin{vmatrix} q \end{vmatrix}\begin{vmatrix} d \end{vmatrix}}$$
   
    This cosine similarities only consider how the two vectors' directions are similar. This is useful since usually the scala value of the query vector is different from the document vector.



In [206]:
import time
import json
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import spacy
from scipy.sparse.linalg import svds, eigs
from spacy.lang.de.stop_words import STOP_WORDS

In [207]:
%run src/file_utils.py
%run src/configuration.py
%run 'load_and_prepro_document.ipynb'

## Collect 1000 documents

In [208]:
# here just use os lib to get the 1000 documents in this folder
import os
documents_list = list()
for root, dirs, files in os.walk(FILE_PATH, topdown=False):
    for name in files:
        documents_list.append(name)
documents_list = documents_list[:1000]

## Preprocess documents, and build TF-IDF matrix based on these documents

In [209]:
start_time = time.time()
# here I override the preProcess() in fit_transform(). Because the input data is already preprocessed.
def preProcess(s):
    return s
my_doc, my_doc_name = get_clean_data(documents_list)
vectorizer = TfidfVectorizer(preprocessor=preProcess)
tfidf_matrix = vectorizer.fit_transform( my_doc )
print (time.time() - start_time)
print (f"The number of input documents is {len(my_doc)}, the shape of tfidf matrix is {tfidf_matrix.shape}")


7.361062288284302
The number of input documents is 950, the shape of tfidf matrix is (950, 233447)


## Input query

In [210]:
query = 'Sportbekleidung schuh '   #query string

## Preprocess the query

In [211]:
# step 1, do same preprosseing for this query
nlp = spacy.load("de")
sentence = nlp(query, disable=['parser', 'ner'])
filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
filtered_words_withoutdigits = [word for word in filtered_words if not word.is_digit]
filtered_words_withoutpunc = [word for word in filtered_words_withoutdigits if word.pos_ != 'PUNCT']
filtered_lemmas = [word.lemma_ for word in filtered_words_withoutpunc]

vocabularly = set()
for word in filtered_lemmas:
    vocabularly.add(word.replace('\n', '').strip().lower())

new_vocab = set()
for u in vocabularly:
    if u != '':
        new_vocab.add(u)

## Generate query's TF-IDF vector

In [212]:
# step 2, generate query's tf-idf vector
query_vector_ori = np.zeros(tfidf_matrix.shape[1]) #initilize the query vector
idf = vectorizer.idf_
feature_name = vectorizer.get_feature_names()

# find my words in this feature_name list, and its corresponding index
print("search query is: ")
print(new_vocab)
for words in new_vocab:
    idx = feature_name.index(words)
    query_vector_ori[idx] = idf[idx]
# do normalize
query_vector_ori = query_vector_ori/np.linalg.norm(query_vector_ori)


search query is: 
{'sportbekleidung', 'schuh'}


## Compute low dimension approximation of the TF-IDF matirx

In [213]:
# step3, transfer the origin tf-idf matrix to low_dim space
k = 100
u, s, vt = svds(tfidf_matrix.T, k=k)  # transpose the tfidf_matrix, get item*document
# here k is the remaining dimension. could from 1 to (number of document-1)
# d_hat = s.inv*U.t*d    
s_dig = np.diag(s)
query_vector_low_dim = ((np.linalg.inv(s_dig)).dot(u.T)).dot(query_vector_ori)
# get query in low dim

## Compute similarity

In [214]:
# step4, compute the similarity
def calculate_similarity(q1,q2):
    sim = q1.dot(q2)/(np.linalg.norm(q1)*np.linalg.norm(q2))
    return sim

sim = np.zeros(vt.shape[1])
for i in range(0,vt.shape[1]):
    sim[i] = calculate_simility(query_vector_low_dim,vt[:,i])

## Show top 10 documents which are similar to the input query

In [215]:
# step5, take top 10 similar document
top_idx = np.argsort(-sim)[0:10]  # here -sim, since I want t get decending order sort,and get the top 3 index
print('------------------------------------')
print('related document: \t related score')
for i in top_idx:
    print(my_doc_name[i]+':\t'+ str(sim[i]))

------------------------------------
related document: 	 related score
PUMA-QuarterlyReport-2012-Q3.json:	0.9705994734434994
PUMA-QuarterlyReport-2012-Q2.json:	0.9585080602136418
PUMA-QuarterlyReport-2015-Q1.json:	0.9544564275740416
PUMA-QuarterlyReport-2014-Q3.json:	0.9507846869303268
PUMA-QuarterlyReport-2010-Q2.json:	0.943183614004445
PUMA-QuarterlyReport-2011-Q3.json:	0.9305251076514407
PUMA-QuarterlyReport-2010-Q1.json:	0.9128798788337482
PUMA-AnnualReport-2013.json:	0.8811760861900734
Adidas-AnnualReport-2016.json:	0.30320260536883675
Zalando-AnnualReport-2015.json:	0.21879978741533074


The top 10 similar (or related) documents I got from this program are 8 PUMA and 1 Addidas and 1 Zalando, which all sell sportswear and shoes. This result shows that this program works. The keypoint here is that I actually do not know if these 2 words occur in these top 10 documents, but LSI technique can still show me these highly related result. This is impossible with traditional key word search.